In [2]:
import minerl
import os
import coloredlogs
import logging
import numpy as np
import tqdm

In [3]:
os.environ['MINERL_DATA_ROOT'] = os.path.expanduser('~/data/minerl')
dat = minerl.data.make('MineRLTreechopVectorObf-v0')

act_vectors = []
for _, act, _, _,_ in tqdm.tqdm(dat.batch_iter(2, 32, 2, preload_buffer_size=2)):
    act_vectors.append(act['vector'])
    if len(act_vectors) > 1000:
        break

0it [00:00, ?it/s]
100%|██████████| 209/209 [00:00<00:00, 13856.59it/s]
913it [00:09, 100.97it/s]


In [4]:
acts = np.concatenate(act_vectors).reshape(-1, 64)

In [5]:
kmeans_acts = acts[:100000]

In [6]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=30, random_state=0).fit(kmeans_acts)

In [7]:
y = kmeans.predict(kmeans_acts)

In [20]:
m = np.r_[True,y[:-1]!=y[1:],True]
counts = np.diff(np.flatnonzero(m))
unq = y[m[:-1]]
repeats = np.c_[unq,counts]

6.100271002710027

In [13]:
print("Clusters")
import pprint
vecs= []
for i in range(30):
    pprint.pprint(minerl.herobraine.envs.MINERL_TREECHOP_OBF_V0.unwrap_action(
    {'vector':  kmeans.cluster_centers_[i]}))
    vecs.append(
        ({'vector':  kmeans.cluster_centers_[i]}, 
int(repeats[np.where(repeats[:,0] == 1)][:,1].mean()))
    )

Clusters
OrderedDict([('attack', array(1)),
             ('back', array(0)),
             ('camera', array([0.0448781 , 0.08084577], dtype=float32)),
             ('forward', array(0)),
             ('jump', array(0)),
             ('left', array(0)),
             ('right', array(0)),
             ('sneak', array(0)),
             ('sprint', array(0))])
OrderedDict([('attack', array(0)),
             ('back', array(0)),
             ('camera', array([0.03759082, 0.20601694], dtype=float32)),
             ('forward', array(0)),
             ('jump', array(0)),
             ('left', array(0)),
             ('right', array(0)),
             ('sneak', array(0)),
             ('sprint', array(0))])
OrderedDict([('attack', array(0)),
             ('back', array(0)),
             ('camera', array([0.04614964, 0.294778  ], dtype=float32)),
             ('forward', array(1)),
             ('jump', array(0)),
             ('left', array(0)),
             ('right', array(0)),
             ('sneak

In [17]:
np.save('vecs', vecs)